<a href="https://colab.research.google.com/github/Konstantin1231/Semester_proj/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers peft evaluate torch numpy datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 991.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━

In [4]:
# Imports
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch #pytorch
import numpy as np


In [6]:
# model choice
model_chekpoint = "distilbert-base-uncased"

#labeling
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# generate model (for classification) additional layer on top to do the classification
model = AutoModelForSequenceClassification.from_pretrained(model_chekpoint, num_labels=2, id2label = id2label, label2id=label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# load dataset
dataset = load_dataset("shawhin/imdb-truncated")
dataset
"""
DatasetDict({
  train: Dataset({
    features: ["label", "text"],
    num_rows: 1000
  })
  validation: Dataset({
    SAME
  })
  })
"""

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

'\nDatasetDict({\n  train: Dataset({\n    features: ["label", "text"],\n    num_rows: 1000\n  })\n  validation: Dataset({\n    SAME\n  })\n  })\n'

**Preprocess Data**


In [11]:
# Tokenaizer
tokenizer = AutoTokenizer.from_pretrained(model_chekpoint, add_prefix_space=True)

#create tokenize function
def tokenize_function(examples):
  #extract text
  text = examples["text"]

  #truncate
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors="np",
      truncation=True,
      max_length = 512
  )
  return tokenized_inputs

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
# add padding for shorter texts
#by default there is no padding, and we should provide it

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({"pad_token": "[PAD]"})
  model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched = True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [13]:
# We need to pad data, to the longest sequence in the batch.

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# Evaluation functiom

accuracy = evaluate.load("accuracy")

# Evaluation function to pass into the trainer

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [25]:
#check model performance
text = "It was kind of ok"

input = tokenizer.encode(text, return_tensors="pt")

#compute logits
logits = model(input).logits
#convert to labels
prediction=torch.argmax(logits)
prediction.tolist()

1

In [28]:
# LORA FOR FINE-TUNNING

peft_config = LoraConfig(task_type="SEQ_CLS",
                         r = 4,
                         lora_alpha=32, #like learning rate
                         lora_dropout=0.01,
                         target_modules = ["q_lin"] # apply LoRA to query layer
)

In [29]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [33]:
# HYPERPARAMETERS
lr = 1e-3
batch_size = 4
num_epochs = 10

# Training argumets
training_args = TrainingArguments(
    output_dir = model_chekpoint + "-lora-cls",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay=0.01,
    evaluation_strategy = "epoch",
    save_strategy= "epoch",
    load_best_model_at_end = True,
)

In [34]:
# Define Trainer

trainer=Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# New Section